## **Lab09 ReLU**

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms 
import random

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)  # seed 가 다른 이유는? 없을 것 같긴 함
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


In [5]:
# paremeters
learning_rate = 0.05
training_epochs = 15
batch_size = 100

In [9]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/', 
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/', 
                          train = False,
                          transform = transforms.ToTensor(),
                          download = True)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1635217266490/work/torch/csrc/utils/tensor_numpy.cpp:189.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [15]:
# dataset loader 
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                          batch_size = batch_size,
                                          shuffle = True, 
                                          drop_last = True)

In [16]:
## nn layers
linear1 = torch.nn.Linear(784, 256, bias = True)
linear2 = torch.nn.Linear(256, 256, bias = True)
linear3 = torch.nn.Linear(256, 10, bias = True)

relu = torch.nn.ReLU()

In [17]:
# Initialization
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[ 1.2744, -0.2283,  0.5615,  ..., -1.4579,  0.2857,  0.1069],
        [-0.4800, -0.7500,  0.4556,  ..., -0.3499,  0.5330, -0.9904],
        [ 0.4220,  0.3790, -0.4462,  ..., -0.4504,  1.9420, -2.4233],
        ...,
        [ 0.4200,  0.5902,  0.9374,  ..., -1.0119,  1.4265,  1.2074],
        [-0.5831,  0.1772, -1.3222,  ...,  0.4466, -1.7848, -1.2168],
        [ 1.0045,  1.3491,  1.4430,  ...,  0.0747, -1.8939, -1.7767]],
       requires_grad=True)

In [18]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device) # linear3는 모델 설정 X


In [19]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device) # SoftMax is internally computed
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # instead SGD, use Adam
 

In [20]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    # reshape input image into [batch_size by 784]
    # label is not one-hot encoded
    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    print('Epoch: ', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))

print('Learning finished')

    

Epoch:  0001 cost =  34.932590485
Epoch:  0002 cost =  0.605318427
Epoch:  0003 cost =  0.361820579
Epoch:  0004 cost =  0.306469709
Epoch:  0005 cost =  0.287957221
Epoch:  0006 cost =  0.286441535
Epoch:  0007 cost =  0.310490072
Epoch:  0008 cost =  0.306537181
Epoch:  0009 cost =  0.331108660
Epoch:  0010 cost =  0.576621294
Epoch:  0011 cost =  0.949737012
Epoch:  0012 cost =  0.909306049
Epoch:  0013 cost =  0.919455826
Epoch:  0014 cost =  1.127674937
Epoch:  0015 cost =  1.358000278
Learning finished


In [21]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r: r+1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r: r+1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy:  0.551800012588501
Label:  9
Prediction:  9


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


왜 낮게 나오는 지 잘 모르겠음